In [ ]:
import json
import requests
import hashlib
import pandas as pd
import glob

from config_key import pub_key, pri_key

In [ ]:
def get_num_val(df_t):
    num_col=['comics','series','stories','events']
    
    for val_col in num_col:
        tmp_col_data = df_t[val_col].apply(lambda x: x['available'])
        df_t[val_col] = tmp_col_data
        
    return df_t

def chk_sts_req(sts_cde,ok_rslt,url_req):
    # check status for the request
    if(sts_cde == 200 and ok_rslt == True):
        print('send a request status from {}: pass'.format(url_req))
    else:
        print('send a request status from {}: fail'.format(url_req))
        print(sts_cde,ok_rslt)
        
def get_data(url_req):
    response_req=requests.get(url_req)
    chk_sts_req(response_req.status_code, response_req.ok,url_req)
    
    return response_req

def save_json_file(response_req,set_num):
    save_sts=0
    
    f_nm_tmp="marvel_char_set_{}.json".format(set_num)
    result_marvel_all = json.loads(response_req.text)
    result_marvel_data = result_marvel_all['data']
  
    if (len(result_marvel_data['results']) != 0):
        with open(src_pth+f_nm_tmp, 'w') as f:
            json.dump(result_marvel_data['results'], f) 
        save_sts=1
    else:
        save_sts=0
        
    return save_sts

def get_lst_nm():
    offset=0
    i=0
    while True:
        save_json_sts=0
        if i > 0:
            offset = offset+int(limit)+1
            url="https://gateway.marvel.com:443/v1/public/characters?offset={}&limit={}&ts={}&apikey={}&hash={}" \
            .format(offset,limit,ts,pub_key,f_hsh_key)
            response = get_data(url)
            save_json_sts=save_json_file(response,i)
            offset = offset-1

        else:
            url="https://gateway.marvel.com:443/v1/public/characters?offset={}&limit={}&ts={}&apikey={}&hash={}" \
            .format(offset,limit,ts,pub_key,f_hsh_key)
            response = get_data(url)
            save_json_sts=save_json_file(response,i)

        if save_json_sts == 0:
            print('done')
            break

        i+=1

In [ ]:
ts="2"
limit="50"
all_key=ts+pri_key+pub_key
hsh_key=hashlib.md5(all_key.encode())
f_hsh_key=hsh_key.hexdigest()

# source file path
src_pth="<enter_own_path>"

In [ ]:
# send requests to get information as multiple json files
get_lst_nm()

# get all json file name from specfic directory
json_files = glob.glob(src_pth+'*.json')

# read json file as dataframe
dt_marvel = pd.concat([pd.read_json(f) for f in json_files])
# drop column
dt_marvel = dt_marvel.drop(['modified','thumbnail','resourceURI','urls'], axis=1)

dt_marvel = get_num_val(dt_marvel)

In [ ]:
dt_marvel